In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from googletrans import Translator
from pathlib import Path
import csv
import os

In [114]:
df03 = pd.read_csv(r'D:\emissionsmap\SubirData\Normalizados\Normal_primary_sub_energy_source.csv')

In [169]:
df01 = pd.read_csv(r'D:\emissionsmap\SubirData\archivosPrueba\huella_biocapacidad.csv')

In [170]:
df01['dif_huel'], df01['dif_biocap'] = df01['Huella Ecologica PerCap'].diff(), df01['Biocapacidad PerCap'].diff()

In [122]:
df_pr = pd.merge(df_n, df01[['year','Huella Ecologica PerCap', 'Biocapacidad PerCap']], on=['year'], how='left')

In [123]:
df_pr.set_index('year', inplace=True)

In [111]:
df_pr.fillna(0, inplace=True)

In [119]:
df_pru = df_n.set_index('year')

In [195]:
def lineal2Plot():
      df01 = pd.read_csv(r'D:\emissionsmap\SubirData\archivosPrueba\huella_biocapacidad.csv')
      fig= go.Figure(data=[
            go.Scatter(
            x=df01['Año'],
            y=df01['Biocapacidad PerCap'],
            mode='lines', 
            name='Biocapacidad',
            line=dict(color='cyan')
            ),
            go.Scatter(
            x=df01['Año'],
            y=df01['Huella Ecologica PerCap'],
            mode='lines',
            name='Huella Ecologica'
            )
      ])
      fig.update_layout(
            # title=name_title,
            height=300,
            plot_bgcolor='black',
            paper_bgcolor= 'black',
            font_color='#cee3e1',
            legend=dict(
                  x=0.05,
                  y=1,
                  title_font_family="Times New Roman",
                  font=dict(
                  family="Courier",
                  size=12,
                  color="LightSteelBlue"
                  ),
                  bgcolor="Black",
                  bordercolor="LightSteelBlue",
                  borderwidth=1
            ),
            xaxis=dict(showgrid=False,showline=True,linecolor='rgb(255,255,255)'),
            yaxis=dict(showgrid=False),
            margin=dict(l=10,r=10,b=10,t=10)
      )
      return fig

In [196]:
lineal2Plot()

In [179]:
df02 = pd.read_csv(r'D:\emissionsmap\SubirData\archivosPrueba\Temperatura_anomalia.csv', sep=';', low_memory=False, encoding='utf-8')
df04 = pd.read_csv(r'D:\emissionsmap\SubirData\archivosPrueba\gases.csv')

In [ ]:
df04.rename(columns={'year':'Año'}, inplace=True)
df04 = df04.sort_values('Año')
df04['Dif_co2'] = df04['Media co2'].diff()
df04 = df04[['Año', 'Dif_co2']]
df02 = df02[['Año', 'Anomalia de la temperatura media del promedio de 1961 a 1990']]
df_n = pd.merge(df02, df04[['Año' ,'Dif_co2']], on=['Año'], how='left')
df_n = df_n[df_n['Año'] >= 1960]

In [197]:
def lineal2Plot2():
    df02 = pd.read_csv(r'D:\emissionsmap\SubirData\archivosPrueba\Temperatura_anomalia.csv', sep=';', low_memory=False, encoding='utf-8')
    df04 = pd.read_csv(r'D:\emissionsmap\SubirData\archivosPrueba\gases.csv')
    
    df04.rename(columns={'year':'Año'}, inplace=True)
    df04 = df04.sort_values('Año')
    df04['Dif_co2'] = df04['Media co2'].diff()
    df04 = df04[['Año', 'Dif_co2']]
    df02 = df02[['Año', 'Anomalia de la temperatura media del promedio de 1961 a 1990']]
    df_n = pd.merge(df02, df04[['Año' ,'Dif_co2']], on=['Año'], how='left')
    df_n = df_n[df_n['Año'] >= 1960]
    
    fig= go.Figure(data=[
        go.Scatter(
        x=df_n['Año'],
        y=df_n['Dif_co2'],
        mode='lines', 
        name='Varición CO2',
        line=dict(color='cyan')
        ),
        go.Scatter(
        x=df_n['Año'],
        y=df_n['Anomalia de la temperatura media del promedio de 1961 a 1990'],
        mode='lines',
        name='Variación Temperatura'
        )
      ])
    fig.update_layout(
        # title=name_title,
        height=300,
        plot_bgcolor='black',
        paper_bgcolor= 'black',
        font_color='#cee3e1',
        legend=dict(
                x=0.05,
                y=1,
                title_font_family="Times New Roman",
                font=dict(
                family="Courier",
                size=12,
                color="LightSteelBlue"
                ),
                bgcolor="Black",
                bordercolor="LightSteelBlue",
                borderwidth=1
        ),
        xaxis=dict(showgrid=False,showline=True,linecolor='rgb(255,255,255)'),
        yaxis=dict(showgrid=False),
        margin=dict(l=10,r=10,b=10,t=10)
    )
    return fig

In [198]:
lineal2Plot2()

In [129]:
df_pr2 = df_pr[['Huella Ecologica PerCap', 'Biocapacidad PerCap']]

In [126]:
df05 = pd.read_csv('D:\emissionsmap\documentation\data\huella_sostenible_por_pais.csv', sep=';')

In [151]:
df_06 = pd.read_csv(r'D:\emissionsmap\SubirData\archivosPrueba\energyco2.csv')

In [3]:
df_06.drop(columns='Unnamed: 0', inplace=True)

In [303]:
df_top = df_06[(df_06['Country'] != 'World') & (df_06['Energy_type'] != 'all_energy_types')]

In [237]:
top_four = df_06[(df_06['Country'] != 'World') & (df_06['Energy_type'] == 'all_energy_types')]

In [201]:
top_four = top_four.groupby('Country')[['Energy_consumption' ,'Energy_production' ,'GDP' , 'Population', 'Energy_intensity_per_capita',
                                        'Energy_intensity_by_GDP', 'CO2_emission']].sum()
top_four.sort_values('CO2_emission', ascending=False, inplace=True)
top_four= top_four.reset_index()
df_top = df_top.groupby(['Country', 'Energy_type'])['CO2_emission'].mean()

In [209]:
types = ['all_energy_types', 'coal', 'natural_gas', 'petroleum_n_other_liquids', 'nuclear', 'renewables_n_other']
top_cinco = top_four['Country'].head(5).to_list()

In [118]:
df_ba = pd.read_csv('D:\emissionsmap\documentation\data\energy.csv')

In [129]:
df_ba = df_ba[(df_ba['Country']=='World') & (df_ba['Energy_type']!='all_energy_types')]
df_ba.drop(columns='Unnamed: 0', inplace=True)

In [131]:
df_ba.head(2)

,Country,Energy_type,Year,Energy_consumption,Energy_production,GDP,Population,Energy_intensity_per_capita,Energy_intensity_by_GDP,CO2_emission,color
1,World,coal,1980,78.656134,80.114194,27770.910281,4.298127e+06,68.145921,10.547,1409.790188,"rgb(116, 255, 70)"
2,World,natural_gas,1980,53.865223,54.761046,27770.910281,4.298127e+06,68.145921,10.547,1081.593377,"rgb(250, 162, 57)"


In [113]:
from raceplotly.plots import barplot

In [ ]:
df_06.drop(columns='Unnamed: 0', inplace=True)

In [148]:
#df:D:\emissionsmap\SubirData\archivosPrueba\energyco2.csv
#col1:'Energy_type' / col2:'Energy_consumption' / colTime:'Year'

def tipoenerW (df, col1, col2, colTieme):
      df = df[(df['Country']=='World') & (df['Energy_type']!='all_energy_types')]
      df.drop(columns='Unnamed: 0', inplace=True)
      my_raceplot = barplot(df,  item_column=col1, value_column=col2, time_column=colTieme,top_entries=10)
      fig=my_raceplot.plot(item_label = col1, value_label = col2, frame_duration = 200, date_format='%Y',orientation='horizontal')

      #Add chart title, format the chart, etc.
      fig.update_layout(
            title=col2,
            title_x=0.15,
            width=800,
            height=550,
            paper_bgcolor="whitesmoke",
            )
      return fig

sandybrown, seagreen, seashell, sienna, silver,
            skyblue, slateblue, slategray, slategrey, snow,
            springgreen, steelblue, tan, teal, thistle, tomato,
            turquoise, violet, wheat, white, whitesmoke,
            yellow, yellowgreen

In [147]:
my_raceplot = barplot(df_ba,  item_column='Energy_type', value_column='Energy_consumption', time_column='Year',top_entries=10)
fig=my_raceplot.plot(item_label = 'Tipo de Energías', value_label = 'Energy_consumption', frame_duration = 200, date_format='%Y',orientation='horizontal')

      #Add chart title, format the chart, etc.
fig.update_layout(
        title='Production de energía',
        title_x=0.15,
        width=800,
        height=550,
        paper_bgcolor="whitesmoke",
            )
